In [1]:
import preprocessing as prep
import numpy as np
import pandas as pd
import nltk
import string
import pickle
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
%matplotlib inline
np.random.seed(42)

In [2]:
df = pd.read_csv('../data/raw.csv', encoding='latin1')

In [3]:
df.columns = ['tweet', 'product', 'emotion']

In [4]:
df = df[df['emotion']!="I can't tell"]
df.dropna(axis=0, subset=['tweet'], inplace=True)

In [5]:
X = df['tweet']
y = df['emotion']

In [6]:
df = pd.concat([X,y],axis=1)

In [7]:
df

,tweet,emotion
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Positive emotion
...,...,...
9088,Ipad everywhere. #SXSW {link},Positive emotion
9089,"Wave, buzz... RT @mention We interrupt your re...",No emotion toward brand or product
9090,"Google's Zeiger, a physician never reported po...",No emotion toward brand or product
9091,Some Verizon iPhone customers complained their...,No emotion toward brand or product


In [23]:
X_train, X_test, y_train, y_test = prep.split(df, int(len(df)*.15))
X_train, X_train_val, y_train, y_train_val = prep.split(pd.concat([X_train, y_train], axis=1), int(len(df)*.15))

In [39]:
X_train = X_train.reset_index()['tweet']
X_train_val = X_train_val.reset_index()['tweet']
X_test = X_test.reset_index()['tweet']

In [40]:
y_train = y_train.reset_index()['emotion']
y_train_val = y_train_val.reset_index()['emotion']
y_test = y_test.reset_index()['emotion']

In [41]:
X_train = list(map(prep.preprocess, list(X_train)))
X_train_val = list(map(prep.preprocess, list(X_train_val)))
X_test = list(map(prep.preprocess, list(X_test)))

In [53]:
X_train_tokenized = list(map(word_tokenize, X_train))
X_train_val_tokenized = list(map(word_tokenize, X_train_val))
X_test_tokenized = list(map(word_tokenize, X_test))

In [54]:
w2v_model = Word2Vec(X_train_tokenized, size=100, window=5, min_count=1, workers=4)

In [55]:
w2v_model.wv.most_similar(positive='apple')

[('shop', 0.9994677305221558),
 ('pop', 0.9987542629241943),
 ('open', 0.998483419418335),
 ('temp', 0.997148334980011),
 ('core', 0.996615469455719),
 ('congress', 0.9950997829437256),
 ('dt', 0.9950422644615173),
 ('set', 0.9949186444282532),
 ('confirmed', 0.9947438836097717),
 ('austin', 0.9939475059509277)]

In [56]:
X_train_w2v = prep.w2v_vectorize(w2v_model.wv, X_train_tokenized)
X_train_val_w2v = prep.w2v_vectorize(w2v_model.wv, X_train_val_tokenized)
X_test_w2v = prep.w2v_vectorize(w2v_model.wv, X_test_tokenized)

KeyError: "word 'jailbreak' not in vocabulary"